In [84]:
!python journal_testing.py

In [85]:
def relative_path(path):
    return ''.join(path.split('/foo', 1)[1:]).strip('/')

In [193]:
def filter_journals(server_journal, client_journal):
    df = pd.read_csv(server_journal, index_col=[0]) # , index_col=[0]
    df1 = pd.read_csv(client_journal, index_col=[0])

    fn = pd.concat([df,df1],ignore_index=True)

    fn['rel'] = fn['src'].apply(lambda path: ''.join(path.split('/foo', 1)[1:]).strip('/'))
    exp = fn.loc[fn.groupby(['rel']).timestamp.idxmax()]

    # SAVE IF NEEDED

    ## exp.to_csv('res.csv')
    return exp.drop(['rel'], axis=1)

In [194]:
tst = filter_journals('tst0.csv', 'tst1.csv')

In [195]:
tst

,timestamp,event,src,dest
4,1537454051,moved,/foo/bar0,/foo/bar9
52,1537454063,moved,/foo/bar1,/foo/bar8
21,1537454078,moved,/foo/bar2,/foo/bar8
31,1537454067,deleted,/foo/bar3,NaN
74,1537454080,modified,/foo/bar4,NaN
38,1537454113,deleted,/foo/bar7,NaN
56,1537454098,deleted,/foo/bar8,NaN
11,1537454072,deleted,/foo/bar9,NaN


In [198]:
def genrtr():
    journal = tst
    for row in journal.iterrows():
        yield row

In [213]:
for row in tst.iterrows():
    print(row[1])
    break

timestamp    1537454051
event             moved
src           /foo/bar0
dest          /foo/bar9
Name: 4, dtype: object


In [110]:
tst.iloc[0:0]

In [58]:
import os
os.path.join(os.path.expanduser('~'), '.recon')

'/home/andretri7/.recon'

In [59]:
import paramiko
import configparser

In [78]:
def get_server_object(server_name, servers_ini):
# 	print (f'\tConnecting with Host: {server_name}')
	host = servers_ini[server_name]['host']
	uname = servers_ini[server_name]['uname']
	port = servers_ini[server_name]['port']
	pkey = servers_ini[server_name]['pkey']

	ssh = paramiko.SSHClient()
	ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
	ssh_pkey = paramiko.RSAKey.from_private_key_file(pkey)

	try:
		ssh.connect(host, port=port, pkey=ssh_pkey, username=uname)
# 		print (f'[+] Success for host: {server_name}')
		_, stdout, _ = ssh.exec_command('echo $HOME')
		
		curr_server = {}
		curr_server = {'connection':ssh, 'host':host, 'uname':uname, 'port':port, 
					   'pkey':pkey, 'recon_path': os.path.join(stdout.readlines()[0].strip('\n'), '.recon')}  
		return curr_server
	except:
# 		print(f"[-] Host {server_name} is Unavailable.")
		return 0		

In [70]:
servers_ini = configparser.ConfigParser()
servers_ini.read('servers.ini')

['servers.ini']

In [73]:
def test():
    srv = get_server_object('VIRTUALBOX', servers_ini)

In [80]:
%timeit test()

355 ms ± 6.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
sftp_client = srv['connection'].open_sftp()
with sftp_client.open('/home/andretri/success.csv', 'r') as f:
    df = pd.read_csv(f)

In [139]:
df

,a,b,c
0,foo,bar,baz


In [191]:
with sftp_client.open('/home/andretri/success.csv', 'r') as f:  
    remote_journal = pd.read_csv(f, index_col=[0])

In [192]:
remote_journal

,b
a,


In [190]:
with sftp_client.open('/home/andretri/success.csv', 'w+') as f:
    remote_journal.iloc[0:0].to_csv(f)